In [ ]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 969.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0

In [ ]:
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 32.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [ ]:
!pip install pytesseract

In [ ]:
import easyocr
import cv2
import pytesseract
import tensorflow as tf
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from PIL import Image

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="f77FoHccQ1RtNOK1tFRo")
project = rf.workspace("jakob-awn1e").project("receipt-or-invoice")
version = project.version(2)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Receipt-or-Invoice-2 in folder:: 100%|██████████| 4325/4325 [00:01<00:00, 4060.46it/s]


In [ ]:
# Preprocessing
def preprocess_images(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image) / 255.0  # Normalization
    return image

In [ ]:
def label_data(data_path):
    images = []
    labels = []
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png'):  # Ensure processing only image files
                file_path = os.path.join(root, file)
                label = os.path.basename(os.path.dirname(file_path))  # Correct label extraction
                image = preprocess_images(file_path)
                images.append(image)
                labels.append(label)
    return np.array(images), np.array(labels)

In [ ]:
# Load dataset
train_path = "/content/Receipt-or-Invoice-2/train"
val_path = "/content/Receipt-or-Invoice-2/valid"
test_path = "/content/Receipt-or-Invoice-2/test"

In [ ]:
X_train, y_train = label_data(train_path)
X_val, y_val = label_data(val_path)
X_test, y_test = label_data(test_path)

In [ ]:
import numpy as np
# Combining labels from all three datasets
all_labels = np.concatenate((y_train, y_val, y_test))

# Encode labels
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Re-encode labels for each dataset
y_train = label_encoder.transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

In [ ]:
# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model
_, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

Epoch 1/10
118/118 [==============================] - 13s 60ms/step - loss: 0.8351 - accuracy: 0.7994 - val_loss: 0.3268 - val_accuracy: 0.8528
Epoch 2/10
118/118 [==============================] - 5s 43ms/step - loss: 0.1771 - accuracy: 0.9308 - val_loss: 0.2239 - val_accuracy: 0.9028
Epoch 3/10
118/118 [==============================] - 5s 43ms/step - loss: 0.1067 - accuracy: 0.9626 - val_loss: 0.3430 - val_accuracy: 0.8750
Epoch 4/10
118/118 [==============================] - 6s 50ms/step - loss: 0.0712 - accuracy: 0.9746 - val_loss: 0.2150 - val_accuracy: 0.9306
Epoch 5/10
118/118 [==============================] - 5s 42ms/step - loss: 0.0466 - accuracy: 0.9822 - val_loss: 0.1966 - val_accuracy: 0.9333
Epoch 6/10
118/118 [==============================] - 5s 44ms/step - loss: 0.0246 - accuracy: 0.9926 - val_loss: 0.5102 - val_accuracy: 0.8611
Epoch 7/10
118/118 [==============================] - 5s 46ms/step - loss: 0.0216 - accuracy: 0.9934 - val_loss: 0.2525 - val_accuracy: 0.927

In [ ]:
# Perform text recognition using OCR
def perform_ocr(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    return result

In [ ]:
# Integrate everything
def integrate_ocr_with_model(image_path):
    # Predict class using the CNN model
    image = preprocess_images(image_path)
    prediction = np.argmax(model.predict(np.array([image])))

    # Perform OCR on the image
    ocr_result = perform_ocr(image_path)

    return label_encoder.classes_[prediction], ocr_result

In [ ]:
# Example usage:
image_path = "/content/Receipt.jpg"
predicted_class, ocr_result = integrate_ocr_with_model(image_path)
print("Predicted Class:", predicted_class)
print("OCR Result:", ocr_result)

1/1 [==============================] - 0s 436ms/step


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Predicted Class: receipt
OCR Result: [([[465, 203], [705, 203], [705, 263], [465, 263]], 'Ber qhote]', 0.6325335589100165), ([[388, 254], [552, 254], [552, 304], [388, 304]], 'Grosse', 0.9941371848520765), ([[568, 252], [684, 252], [684, 308], [568, 308]], 'Sche', 0.9995225071907043), ([[439, 301], [727, 301], [727, 345], [439, 345]], '3818 Gr indelwa Id', 0.6502805743947839), ([[439, 349], [727, 349], [727, 393], [439, 393]], 'Fami ]ie R. Mul ler', 0.33013049470401395), ([[179, 447], [313, 447], [313, 489], [179, 489]], 'Rech. Nr', 0.6187506710158083), ([[335, 447], [417, 447], [417, 489], [335, 489]], '4572', 0.6993084426273128), ([[594, 446], [932, 446], [932, 494], [594, 494]], '30.07.2007/13:29:17', 0.619726702847209), ([[179, 495], [241, 495], [241, 537], [179, 537]], 'Bar', 0.9488676837365144), ([[717, 495], [813, 495], [813, 539], [717, 539]], 'Tisch', 0.9999296278479316), ([[854, 494], [932, 494], [932, 542], [854, 542]], '7/01', 0.9999768733978271), ([[195, 593], [503, 593], 